In [1]:
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split

from lib.Utility import exportExcelWithTimeStamp

from datetime import date, timedelta
import yfinance as yf #Alternative package if webreader does not work: pip install yfinance
import numpy as np # Fundamental package for scientific computing with Python
import joblib
import plotly.express as px
from plotly import graph_objects as go
# Train the model
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

# features = ['B365H','B365D', 'B365A',	'B365<2.5']
features = ['HomeGoalsCumulative', 'AwayGoalsCumulative', 'HomePointsCumulative','AwayPointsCumulative', 'HomeGoalsConcededCumulative',	'AwayGoalsConcededCumulative'
            #  , 'B365D'
            #  , 'B365H'
            # , 'B365A'
            # , 'B365>2.5'
            ]
# features = ['HomeGoalsCumulative', 'AwayGoalsCumulative', 'AwayPointsCumulative', 'HomeGoalsConcededCumulative']
# features = ['HomeGoalsCumulative', 'AwayGoalsConcededCumulative']
# features = ['HomeGoalsCumulative', 'HomePointsCumulative']

threshold = 0.2

In [2]:
data = pd.read_excel('data/mergedDataPremierLeague.xlsx')
# data = pd.read_excel('data/engdata/24-25.xlsx')
# data = pd.read_excel('data/mergedDataSerieB.xlsx')
# data['Cluster'] = np.where(data['FTR'] == 'H', 1, np.where(data['FTR'] == 'A', 2, 0))
data['Cluster'] = np.where(data['FTHG'] == data['FTAG'], 1, 0) #data['FTR'].map({'D': 0, 'H': 1, 'A': 2}) 
# data = data[data['HomeGoalsCumulative'].notna()]
# data = data[data['AwayGoalsCumulative'].notna()]
# data = data[data['HomePointsCumulative'].notna()]
# data = data[data['AwayPointsCumulative'].notna()]
# data = data[data['HomeGoalsConcededCumulative'].notna()]
# data = data[data['AwayGoalsConcededCumulative'].notna()]
data = data[data['B365D'].notna()]
# data = data[data['B365H']>1.6]
# data.dropna()
data.tail()

,Date,Div,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgCAHH,AvgCAHA,HomeGoalsCumulative,AwayGoalsCumulative,HomePointsCumulative,AwayPointsCumulative,HomeGoalsConcededCumulative,AwayGoalsConcededCumulative,MatchGoal,Cluster
1895,1970-01-01 01:00:45.431,E0,16:00:00,Crystal Palace,Aston Villa,5,0,H,2,0,...,1.81,2.05,52,76,46,68,58,56,5,0
1896,1970-01-01 01:00:45.431,E0,16:00:00,Liverpool,Wolves,2,0,H,2,0,...,2.04,1.82,84,50,79,46,41,63,2,0
1897,1970-01-01 01:00:45.431,E0,16:00:00,Luton,Fulham,2,4,A,1,2,...,1.96,1.91,50,51,26,44,81,59,6,0
1898,1970-01-01 01:00:45.431,E0,16:00:00,Man City,West Ham,3,1,H,2,1,...,1.96,1.91,93,59,88,52,33,71,4,0
1899,1970-01-01 01:00:45.431,E0,16:00:00,Sheffield United,Tottenham,0,3,A,0,1,...,2.01,1.86,35,71,16,63,101,61,3,0


In [3]:
x_train, x_test, y_train, y_test = train_test_split(data[features], data['Cluster'], test_size= 0.3, random_state= 42,shuffle=True)

x_train = x_train.sort_index()
x_test = x_test.sort_index()
y_train= y_train.sort_index()
y_test = y_test.sort_index()

print (f'X_train: {x_train.shape} \nX_test: {x_test.shape} \ny_train: {y_train.shape} \ny_test: {y_test.shape}')


X_train: (1330, 6) 
X_test: (570, 6) 
y_train: (1330,) 
y_test: (570,)


In [4]:
model = joblib.load("Soccer-ml4.joblib")

In [5]:
def custom_predict(X, threshold):
    probs = model.predict_proba(X) 
    return (probs[:, 1] > threshold).astype(int)
# Definisci una funzione che applica la logica descritta
def calculate_gain(row, quota_minima):
    
    if(row['predictions'] > threshold):
        if row['Cluster'] == row['predictions']:
            if row['predictions'] == 1 and row['B365H'] > quota_minima:
                return row['B365H']-1
            elif row['predictions'] == 2 and row['B365A'] > quota_minima:
                return row['B365A']-1
            elif row['predictions'] == 0 and row['B365D'] > quota_minima:
                return row['B365D']-1
        else:
            if((row['predictions'] == 0 and row['B365D'] > quota_minima) or (row['predictions'] == 2 and row['B365A'] > quota_minima) or (row['predictions'] == 1 and row['B365H'] > quota_minima)):
                return -1

In [6]:
x_test = data[features]
y_test = data['Cluster']

df = x_test.copy()
# df['predictions'] = model.predict(x_test.sort_index())
df['predictions'] = custom_predict(x_test.sort_index(), threshold)
pred_prob = model.predict_proba(x_test.sort_index())

df2 = pd.DataFrame(pred_prob)
df2.index = df.index
df = df.merge(df2, left_index=True, right_index=True)

finalDataset = data[data.index.isin(x_test.index)]
finalDataset['predictions'] = df['predictions']
finalDataset['pred_prob'] = df[1]
# finalDataset['gain'] = finalDataset.apply(calculate_gain, axis=1, args=(1.9,))
finalDataset['gain'] = np.where((finalDataset['FTR'] == 'D') & (finalDataset['predictions'] > threshold),
                                finalDataset['B365D']-1, np.where((finalDataset['FTR'] != 'D') & (finalDataset['predictions'] > threshold), -1, None))

equity = finalDataset['gain'].sum()
print("Equity", equity)

print(f"Accuracy score: {round(100*accuracy_score(y_test, df['predictions']),2)}%")
exportExcelWithTimeStamp(finalDataset, "Dataframe/", "df.xlsx")

Equity -101.84000000000006
Accuracy score: 32.79%
